In [10]:
import { load } from "dotenv";

import { ChatOpenAI } from "@langchain/openai";







const env = await load();






const process = {

  env,

};






const chatModel = new ChatOpenAI({

  configuration: {

    baseURL: "https://api.moonshot.cn/v1",

    apiKey: process.env.OPENAI_API_KEY,

  },

  modelName: "kimi-k2-0711-preview",

});

In [11]:
import { RecursiveCharacterTextSplitter } from "langchain/text_splitter";

import { TextLoader } from "langchain/document_loaders/fs/text";






const loader = new TextLoader("data/kong.txt");

const docs = await loader.load();






const splitter = new RecursiveCharacterTextSplitter({

  chunkSize: 64,

  chunkOverlap: 16,

});






const splitDocs = await splitter.splitDocuments(docs);



In [12]:
splitDocs;


[
  Document {
    pageContent: "鲁镇的酒店的格局，是和别处不同的：都是当街一个曲尺形的大柜台，柜里面预备着热水，可以随时温酒。做工的人，傍午傍晚散了工，每每花四",
    metadata: { source: "data/kong.txt", loc: { lines: { from: 1, to: 1 } } }
  },
  Document {
    pageContent: "工的人，傍午傍晚散了工，每每花四文铜钱，买一碗酒，——这是二十多年前的事，现在每碗要涨到十文，——靠柜外站着，热热的喝了休息；倘",
    metadata: { source: "data/kong.txt", loc: { lines: { from: 1, to: 1 } } }
  },
  Document {
    pageContent: "—靠柜外站着，热热的喝了休息；倘肯多花一文，便可以买一碟盐煮笋，或者茴香豆，做下酒物了，如果出到十几文，那就能买一样荤菜，但这些",
    metadata: { source: "data/kong.txt", loc: { lines: { from: 1, to: 1 } } }
  },
  Document {
    pageContent: "十几文，那就能买一样荤菜，但这些顾客，多是短衣帮，大抵没有这样阔绰。只有穿长衫的，才踱进店面隔壁的房子里，要酒要菜，慢慢地坐喝。",
    metadata: { source: "data/kong.txt", loc: { lines: { from: 1, to: 1 } } }
  },
  Document {
    pageContent: "我从十二岁起，便在镇口的咸亨酒店里当伙计，掌柜说，我样子太傻，怕侍候不了长衫主顾，就在外面做点事罢。外面的短衣主顾，虽然容易说",
    metadata: { source: "data/kong.txt", loc: { lines: { from: 3, to: 3 } } }
  },
  Document {
    pageContent: "事罢。外面的短衣主顾，虽然容易说话，但唠唠叨叨缠夹不清的也很不少。他们往往要亲眼看着黄酒从坛子里舀出，看过壶子底里有水没有，又

In [18]:
import { OllamaEmbeddings } from "npm:/@langchain/ollama@0.1.5";

import { RecursiveCharacterTextSplitter } from "langchain/text_splitter";

import { TextLoader } from "langchain/document_loaders/fs/text";







const loader = new TextLoader("data/kong.txt");

const docs = await loader.load();






const splitter = new RecursiveCharacterTextSplitter({

  chunkSize: 64,

  chunkOverlap: 16,

});






const splitDocs = await splitter.splitDocuments(docs);

const embeddings = new OllamaEmbeddings({
  model: "deepseek-r1",
  baseUrl: "http://localhost:11434/",
});






const res = await embeddings.embedQuery(splitDocs[0].pageContent);

res

[
    0.021112114,  0.0026643863,  -0.010143283,  0.0010134631,   0.022204274,
   -0.010080873, -0.0047391127,   0.028232314,  0.0068481117,     0.0331705,
   -0.009197335,  -0.025985688,  0.0040287403,   0.037609696,  0.0022131274,
   0.0022850297,  -0.037228078,   0.010380872,  -0.017773058,  -0.013160624,
   -0.006967208, -0.0042489055,   -0.02275206,  0.0058082435,   0.007157981,
    0.013082582,  -0.028171722,   0.012403142,   0.013324624,  0.0026278454,
   -0.037041288,   -0.01856304,   0.008140081,  0.0063545285,  0.0063172523,
    0.017727125, -0.0019789864,   0.006299152,  0.0022036564,   0.023667578,
    0.017638393,  -0.009643243,  -0.009050061, -0.0055772373,   0.017445818,
    0.014429503,   0.031296853,  -0.008588404,  -0.010940751,   0.008833729,
    0.017612621,  -0.010808378,  -0.009219888,   0.001980425,   0.017345436,
   -0.010118796,   0.009340232,  0.0019008068,   -0.01131199,   0.016694065,
   -0.004770128,  -0.014163698,   0.018095512,  0.0012204668,   0.03290450

In [14]:
import {MemoryVectorStore} from 'langchain/vectorstores/memory'

const vectorstore = new MemoryVectorStore(embeddings)
await vectorstore.addDocuments(splitDocs)

In [15]:
const retriever = vectorstore.asRetriever(2)

In [16]:
const res = await retriever.invoke('茴香豆是做什么用的')
res

[
  Document {
    pageContent: "安模样，脸上笼上了一层灰色，嘴里说些话；这回可是全是之乎者也之类，一些不懂了。在这时候，众人也都哄笑起来：店内外充满了快活的空气",
    metadata: { source: "data/kong.txt", loc: { lines: { from: 11, to: 11 } } }
  },
  Document {
    pageContent: "“多乎哉?不多也。”",
    metadata: { source: "data/kong.txt", loc: { lines: { from: 13, to: 13 } } }
  }
]

In [17]:
const res = await retriever.invoke('下酒菜一般是什么')
res

[
  Document {
    pageContent: "“多乎哉?不多也。”",
    metadata: { source: "data/kong.txt", loc: { lines: { from: 13, to: 13 } } }
  },
  Document {
    pageContent: "安模样，脸上笼上了一层灰色，嘴里说些话；这回可是全是之乎者也之类，一些不懂了。在这时候，众人也都哄笑起来：店内外充满了快活的空气",
    metadata: { source: "data/kong.txt", loc: { lines: { from: 11, to: 11 } } }
  }
]